<a href="https://colab.research.google.com/github/DanielaCaroline18/NLP/blob/main/NLP_Aula_3_Similaridade_de_Documentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install afinn
!python -m textblob.download_corpora
!pip install -U textblob
!pip install vaderSentiment
!pip install pyemd

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for afinn: filename=afinn-0.1-cp36-none-any.whl size=53452 sha256=cc9d6c2a57522587ecfc45f00fd3f970ad7065cc73bc019489b97eaf8b8516db
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]

In [ ]:
pip install unidecode

     |████████████████████████████████| 245kB 2.8MB/s 


In [ ]:
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from unidecode import unidecode
import pandas as pd
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

warnings.filterwarnings('ignore')
tqdm_notebook.pandas()

import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  app.launch_new_instance()


# Carregando os embeddings

Aqui vamos utilizar os embeddings para realizar as seguintes atividades:

- análise de simlaridade
- classificação de documentos

<b> Carregue os embeddings treinados, como vimos na Aula 2. É o mesmo arquivo que iremos utilizar</b>

Link: https://drive.google.com/open?id=1zI8pGfbUHuU_0wY_FV4tD6w6ZCUJTQbh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%time
newfilepath = "/content/drive/My Drive/embedding_wiki_100d_pt.txt"
filepath = "/content/drive/My Drive/ptwiki_20180420_100d.txt.bz2"
with open(newfilepath, 'wb') as new_file, bz2.BZ2File(filepath, 'rb') as file:
    for data in iter(lambda : file.read(100 * 1024), b''):
        new_file.write(data)
 


CPU times: user 42.3 s, sys: 1.07 s, total: 43.3 s
Wall time: 46.3 s


In [ ]:
%%time
# carregar
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(newfilepath, binary=False)

CPU times: user 1min 46s, sys: 1.8 s, total: 1min 48s
Wall time: 1min 48s


# Similaridade de Documentos

Para realizar a similaridade entre documentos, utilize as frases abaixo:

In [ ]:
frase1 = "Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado" 
frase2 = "SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!"
frase3 = "A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos"
frase4 = "Fogão ótimo!"

## Distância de Jaccard

<b> Atividade </b>

1) Faça um método que calcule a similaridade de Jaccard e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: lembrando que você precisa aplicar um pre-processamento nessas frases antes de aplicar o método.
Faça:

- Lower
- Remoção StopWords
- Remoção Pontuação
- Tokenização

In [ ]:
 import nltk
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def pre_processamento_texto(corpus):
    print("Documento")
    #tokenizacao"
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #lowcase
    corpus_alt = [t.lower() for t in corpus_alt]
    #remove stopwords
    portugues_stops = stopwords.words('portuguese')
    corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
    #remove pontuação
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]
   
    return corpus_alt

In [ ]:
frase1_pre_processada = pre_processamento_texto(frase1) 
frase2_pre_processada = pre_processamento_texto(frase2) 
frase3_pre_processada = pre_processamento_texto(frase3) 
frase4_pre_processada = pre_processamento_texto(frase4) 

Documento
Documento
Documento
Documento


intersecao / união

In [ ]:
def jaccard_similarity(f1, f2):
  return len(set(f1).intersection(set(f2)))/ len(set(f1).union(set(f2)))

In [ ]:
print ("Frase 1 e Frase 2",jaccard_similarity(frase1_pre_processada, frase2_pre_processada) )
print ("Frase 1 e Frase 3",jaccard_similarity(frase1_pre_processada, frase3_pre_processada) )
print ("Frase 2 e Frase 3",jaccard_similarity(frase2_pre_processada, frase4_pre_processada) )
print ("Frase 1 e Frase 4",jaccard_similarity(frase1_pre_processada, frase3_pre_processada) )

Frase 1 e Frase 2 0.10526315789473684
Frase 1 e Frase 3 0.0
Frase 2 e Frase 3 0.0
Frase 1 e Frase 4 0.0


<b> Atividade </b>

2) Qual par de frase teve maior simlaridade? E qual teve menor? Este resultado faz sentido? Explique.

O par de dados que teve maior similaridade foram as frases 1 e 2. Isso faz sentido pois ambas as frases elogiam o produto e o recomendam . Os pares 1 e 3, e 2 e 3 tiveram similaridade 0, faz sentido pois enquanto em uma das frases o produto é elogiado e recomendado, na outra são levantados alguns defeitos do produto. Já em relação ao par 1 e 4, apesar de ambas falar bem o resultado foi 0, isso se deve ao fato de as frases não conterem palavras iguais e, portanto, a interseção entre elas é nula.

## Distância de Cosseno

Aqui iremos calcular a distância do cosseno utilizando duas formas, que aprendemos na aula passada, para representar o texto.

- Bag of Words (BOW) 
- Embedding

Observação:

Existem duas formas de trabalhar com o cosseno:

<b> Distância </b>: quanto menor mais perto estão as frases.
<b> Similaridade </b>: quanto maior mais perto estão as frases.

### BOW - Distância do cosseno

<b> Atividade </b>

3) Calcule a distância do cosseno utilizando a representação CountVectorizer e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: no CountVectorizer utilizem as frases já pre-processadas da atividade anterior. Mas para aplicá-las no fit_transform, cada frase deve ser um string (sem estar tokenizada) dentro de uma lista.

```python
#exemplo
distance.cosine(frase1, frase2)
```

In [ ]:
vect_bag = CountVectorizer()
vect_bag

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
frases = [' '. join(frase1_pre_processada),
                       ' '. join(frase2_pre_processada),
                       ' '. join(frase3_pre_processada),
                       ' '. join(frase4_pre_processada)]

In [ ]:
len(frases)

4

In [ ]:
docs_bag = vect_bag.fit_transform(frases)

In [ ]:
docs_bag.shape

(4, 29)

In [ ]:
frase_1_bow = docs_bag.todense()[0]
frase_2_bow = docs_bag.todense()[1]
frase_3_bow = docs_bag.todense()[2]
frase_4_bow = docs_bag.todense()[3]

In [ ]:
print ("Frase 1 e Frase 2",distance.cosine(frase_1_bow, frase_2_bow) )
print ("Frase 1 e Frase 3",distance.cosine(frase_1_bow, frase_3_bow) )
print ("Frase 2 e Frase 3",distance.cosine(frase_2_bow, frase_4_bow) )
print ("Frase 1 e Frase 4",distance.cosine(frase_1_bow, frase_3_bow) )

Frase 1 e Frase 2 0.8309691490542968
Frase 1 e Frase 3 1.0
Frase 2 e Frase 3 1.0
Frase 1 e Frase 4 1.0


<b> Atividade </b>


4) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

Da mesma forma do método de Distância de Jaccard o método BOW só encontrou similaridade entre as frases 1 e 2, mas não foi capaz de encontrar similaridade entre as frases 1 e 4 que também é positiva. Portanto, esse assim como a Distância de Jaccard, não foi o melhor meio para se calcular a similaridade entre as frases.

### Embedding - Distância do cosseno

Para calcular o embedding de cada uma das frases, utilize o modelo carregado inicialmente. 

Cada palavra tem um vetor, para formar o embedding da frase tire a média de todos os vetores.

Utilize as frases já pre-processadas

<b> Atividade </b> 

5) Calcule a distância do cosseno utilizando a representação Embedding e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4


In [ ]:
def get_embeddings (frase):  
   return np.mean(np.array([word_vectors[w] for w in frase if w in word_vectors.vocab]), axis=0)

In [ ]:
frase_1_embedding = get_embeddings(frase1_pre_processada)
frase_2_embedding = get_embeddings(frase2_pre_processada)
frase_3_embedding = get_embeddings(frase3_pre_processada)
frase_4_embedding = get_embeddings(frase4_pre_processada)

In [ ]:
np.array(frase_1_embedding).shape

(100,)

In [ ]:
print ("Frase 1 e Frase 2",distance.cosine(frase_1_embedding, frase_2_embedding) )
print ("Frase 1 e Frase 3",distance.cosine(frase_1_embedding, frase_3_embedding) )
print ("Frase 2 e Frase 3",distance.cosine(frase_2_embedding, frase_3_embedding) )
print ("Frase 1 e Frase 4",distance.cosine(frase_1_embedding, frase_4_embedding) )

Frase 1 e Frase 2 0.16096973419189453
Frase 1 e Frase 3 0.2234203815460205
Frase 2 e Frase 3 0.2526938319206238
Frase 1 e Frase 4 0.29043054580688477


<b>Atividade </b>

6) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

Nesse caso, quanto menor a distância mais similar é uma frase da outra. Usando o Embedding podemos ver uma variabilidade muito maior dos resultados comparados aos outros métodos o que nos permite fazer uma análise melhor. De novo é possível confirmar a similaridade entre as frases 1 e 2 devido a menor distância, entretando a frase 4 por ser uma frase muito pequena ao compará-la com a frase 1, apesar de ambas serem positivas o valor da distância é o mais alto, logo são pouco similares.  Ao comparar as frases 1 e 3, e 2 e 3, apesar de serem um positiva e a outra negativa, acredito que como algumas palavras usadas nos dois contextos são próxima a distância do conseno não foi muito grande, impedido que fosse possível contatar que as opiniões expostas pelos usuários são divergentes.

## WMD

O WMD já está incorporado ao Word2Vec

<b> Atividade </b>

7) Calcule a distância WMD e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: use a variável já tokenizada.

In [ ]:
print ("Frase 1 e Frase 2",word_vectors.wmdistance(frase1_pre_processada, frase2_pre_processada) )
print ("Frase 1 e Frase 3",word_vectors.wmdistance(frase1_pre_processada, frase3_pre_processada) )
print ("Frase 2 e Frase 3",word_vectors.wmdistance(frase2_pre_processada, frase3_pre_processada) )
print ("Frase 1 e Frase 4",word_vectors.wmdistance(frase1_pre_processada, frase3_pre_processada) )

Frase 1 e Frase 2 3.0725697404938526
Frase 1 e Frase 3 3.759219170455064
Frase 2 e Frase 3 3.9048993635316385
Frase 1 e Frase 4 3.759219170455064


<b> Atividade </b>

8) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

Avalianndo os valores vemos que as frases que deram mais próximas foram as frases 1 e 2, confirmando o mesmo que os outros métodos acima. A frase mais distante foi para o par de frases 2 e 3, esse método e o que permitiu visualizar uma distância maior entre essas duas frases confirmando a opinião divergente entre elas. Entretando, ao comparar as frases 2 e 3 apesar da distância ser maior indicando baixa similidade, não chega a ser tão expressiva se comparada à distancia entre as frases 1 e 3, mesmo essas tendo opiniões divergentes. Já como esperado, devido os resultados anteriores, como a frase 4 é muito curta impedi que ao calcular a distância o valor demostre a similiridade de sentido entre as frases 1 e 4.

# Classificação de Documentos

A clssificação de documentos é muito útil em vários aspectos. Um dos tipos de classificação de texto é a análise de sentimentos.

A fim de ilustrar a classificação de documentos iremos criar um modelo para classificar uma frase como positiva ou negativa.

## Dataset

<b> Atividade </b>

9) Carregue o dataset com o pandas e depois dê o head no dataframe.


Link download: https://drive.google.com/open?id=15azJWdEEPGsXQGiDmEOseTBJcquWvBQc

<b> Este dataset é sobre revisões de filmes do IMDB. </b>

In [ ]:
filepath = "/content/drive/My Drive/imdb-reviews-pt-br.csv"

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


## Representação dos dados

O sentimento positivo e negativo iremos binarizar cada um deles. Seja 1 positivo e 0 negativo.

Iremos representar o texto de duas formas:

- Bag of Words (BOW)
- Embedding

Depois iremos comparar o resultado de cada um deles.

### Representação Target

<b> Atividade </b>

10) Faça a representação dos sentimentos. 1 positivo; 0 negativo

In [ ]:
target = df["sentiment"].replace(["neg", "pos"], [0, 1])

In [ ]:
target

0        0
1        0
2        0
3        0
4        0
        ..
49454    1
49455    1
49456    1
49457    1
49458    1
Name: sentiment, Length: 49459, dtype: int64

### Bag of Words (BOW)

<b> Atividade </b>

11) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords``` no dataframe para armazenar este dado processado):

- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto sem tokenização, ou seja uma string

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [ ]:
def pre_processamento_texto_return_str(corpus):
    #tokenizacao"
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #remove stopwords
    portugues_stops = stopwords.words('portuguese')
    corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
    #remove pontuação
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]
    #lowercase  //Bárbara usei o Lowercase porque na aula você falou que você tinha esquecido de colocar
    corpus_alt = [t.lower() for t in corpus_alt]

    corpus_alt_str = ' '.join(corpus_alt)
   
    return corpus_alt_str

In [ ]:
df["text_pt_sem_stopwords"] =  df["text_pt"].progress_apply(lambda x: pre_processamento_texto_return_str(x))

In [ ]:
df.head()

,id,text_en,text_pt,sentiment,text_pt_sem_stopwords
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,mais vez sr costner arrumou filme tempo necess...
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,este exemplo motivo maioria filmes ação mesmos...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro tudo odeio raps imbecis poderiam agir...
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,nem beatles puderam escrever músicas todos gos...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,filmes fotos latão palavra apropriada verdade ...


<b> Atividade </b>

12) Aplique a representação do texto processado anteriormente com CountVectorizer.

In [ ]:
docs_bag = vect_bag.fit_transform(df["text_pt_sem_stopwords"])

In [ ]:
docs_bag

<49459x129589 sparse matrix of type '<class 'numpy.int64'>'
	with 5308378 stored elements in Compressed Sparse Row format>

### Embedding

<b> Atividade </b>

13) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords_token``` no dataframe para armazenar este dado processado):

- Aplique lower
- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto com tokenização

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [ ]:
def pre_processamento_texto_return_token(corpus):
    #tokenizacao"
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #lowcase
    corpus_alt = [t.lower() for t in corpus_alt]
    #remove stopwords
    portugues_stops = stopwords.words('portuguese')
    corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
    #remove pontuação
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]
   
    return corpus_alt

In [ ]:
df["text_pt_sem_stopwords_token"] =  df["text_pt"].progress_apply(lambda x: pre_processamento_texto_return_token(x))

In [ ]:
df.head()

,id,text_en,text_pt,sentiment,text_pt_sem_stopwords,text_pt_sem_stopwords_token
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,mais vez sr costner arrumou filme tempo necess...,"[vez, sr, costner, arrumou, filme, tempo, nece..."
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,este exemplo motivo maioria filmes ação mesmos...,"[exemplo, motivo, maioria, filmes, ação, mesmo..."
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro tudo odeio raps imbecis poderiam agir...,"[primeiro, tudo, odeio, raps, imbecis, poderia..."
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,nem beatles puderam escrever músicas todos gos...,"[beatles, puderam, escrever, músicas, todos, g..."
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,filmes fotos latão palavra apropriada verdade ...,"[filmes, fotos, latão, palavra, apropriada, ve..."



<b> Atividade </b>

14) Aplique a representação do texto com Embeddings. Cada palavra tem um embedding, o embedding da frase é a média de todos embeddings.

In [ ]:
X_embedding = df["text_pt_sem_stopwords_token"].progress_apply(lambda x: get_embeddings(x))

## Treinamento

### CountVectorizer


<b> Atividade </b>

15) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

```python
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bag, target,random_state=123)
```

In [ ]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(docs_bag, target,random_state=123)


<b> Atividade </b>

16) Treine com uma regressão logística

In [ ]:
modelo_bow = LogisticRegression()
modelo_bow.fit(X_train_bow, y_train_bow)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


<b> Atividade </b>

17) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

In [ ]:
y_pred=modelo_bow.predict(X_test_bow)

In [ ]:
y_pred

array([1, 1, 1, ..., 0, 1, 1])

In [ ]:
print(classification_report(y_test_bow, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      6112
           1       0.88      0.89      0.88      6253

    accuracy                           0.88     12365
   macro avg       0.88      0.88      0.88     12365
weighted avg       0.88      0.88      0.88     12365



### Embedding


<b> Atividade </b>

18) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

Verifique o shape do X treino e X teste. Caso eles estejam com apenas uma dimensão, você precisa tranformá-los para duas dimensões, caso contrário ocorrerá erro no treinamento.

In [ ]:
X_train_embedding, X_test_embedding, y_train_embedding, y_test_embedding = train_test_split(X_embedding.values, target,random_state=123)

In [ ]:
X_train_embedding = pd.DataFrame([x for x in X_train_embedding])
X_test_embedding = pd.DataFrame([x for x in X_test_embedding])

In [ ]:
X_train_embedding.shape

(37094, 100)

In [ ]:
X_train_embedding

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.164775,0.177243,-0.091046,-0.168941,-0.299286,-0.276219,0.068210,0.171216,-0.085756,0.352760,0.288270,-0.139173,-0.346397,0.028892,0.057119,0.085538,0.197978,-0.140617,0.182710,-0.234322,0.140444,0.197303,-0.259221,0.010419,-0.089338,-0.033548,-0.353268,-0.412356,-0.226751,0.188906,0.070062,0.130738,0.239321,0.258778,0.283559,-0.188337,0.146227,0.128092,0.214179,-0.285062,...,0.430197,-0.290119,-0.310422,0.193344,0.077103,0.065438,-0.332814,0.143340,0.127281,-0.237968,-0.234810,0.105136,-0.279995,-0.376887,-0.051881,-0.108376,-0.237884,0.061548,-0.151421,-0.128395,0.064070,-0.294903,-0.159370,-0.147060,-0.311051,0.160057,0.123889,-0.262308,-0.056302,-0.216241,0.568133,-0.394102,-0.062824,-0.059671,0.165389,-0.146962,0.033940,0.135802,-0.148060,0.259065
1,0.270804,0.055139,-0.072599,-0.117039,-0.194689,-0.306264,0.022104,0.162099,-0.070719,0.425425,0.156085,-0.145835,-0.376195,0.028109,0.033679,0.088126,0.232945,-0.093876,0.220061,-0.182259,0.106641,0.115869,-0.217178,-0.021485,-0.043441,0.018057,-0.215116,-0.254944,-0.226162,0.209804,0.102631,0.083512,0.252391,0.344627,0.199610,-0.250239,0.099688,0.171356,0.283941,-0.350958,...,0.420389,-0.253485,-0.258061,0.182879,0.053920,0.134684,-0.256869,0.056299,0.172810,-0.168708,-0.218559,0.124098,-0.277063,-0.511146,0.006866,-0.136825,-0.238070,0.080749,-0.154237,-0.142029,0.124534,-0.270540,-0.226084,-0.186330,-0.264603,0.165545,0.202721,-0.331068,-0.083111,-0.078711,0.596688,-0.338906,-0.142922,-0.177890,0.177174,-0.034758,-0.013745,0.047016,-0.156159,0.217429
2,0.248587,0.192205,-0.037750,-0.144611,-0.249053,-0.260594,0.077630,0.221875,-0.067744,0.445615,0.218363,-0.142251,-0.370959,-0.038130,-0.013235,0.027577,0.204796,-0.195639,0.129604,-0.221722,0.105359,0.168186,-0.184282,-0.017558,-0.015111,0.021592,-0.358987,-0.315411,-0.187731,0.176140,0.004706,0.113213,0.211111,0.234917,0.239069,-0.210515,0.095759,0.197356,0.243410,-0.306429,...,0.419168,-0.315818,-0.260172,0.257668,-0.080672,0.175380,-0.296332,0.092363,0.084530,-0.282227,-0.246190,0.102043,-0.220269,-0.334704,-0.076154,-0.063929,-0.245101,0.137309,-0.131172,-0.115422,0.089121,-0.240180,-0.146096,-0.171158,-0.265064,0.186290,0.114099,-0.247230,-0.147761,-0.133124,0.570264,-0.324722,-0.076872,-0.139255,0.251056,-0.134613,-0.022701,0.117389,-0.071973,0.282469
3,0.208730,0.206336,-0.094195,-0.162718,-0.250112,-0.306295,0.063157,0.097309,-0.133926,0.386954,0.238586,-0.172071,-0.386691,0.010272,-0.010906,0.017958,0.184571,-0.155206,0.187456,-0.164736,0.077164,0.140951,-0.224388,0.016389,-0.005889,0.015916,-0.353920,-0.402255,-0.185143,0.259346,0.058873,0.163960,0.156002,0.268297,0.312440,-0.190172,0.087351,0.132719,0.218084,-0.256908,...,0.376571,-0.294111,-0.281558,0.220269,0.044552,0.136160,-0.294362,0.095797,0.155663,-0.316048,-0.226740,0.082501,-0.179877,-0.432484,-0.031216,-0.101213,-0.231649,0.057543,-0.171885,-0.054744,0.104736,-0.323152,-0.200952,-0.205490,-0.297817,0.112389,0.176066,-0.266190,-0.106083,-0.111462,0.606852,-0.345584,-0.089162,-0.056111,0.250674,-0.129950,0.002193,0.114857,-0.085246,0.278568
4,0.256833,0.204345,-0.081420,-0.148684,-0.332549,-0.304174,0.051899,0.154746,-0.113199,0.425564,0.274771,-0.186432,-0.420828,-0.050684,0.073438,0.034845,0.242141,-0.133862,0.128874,-0.173321,0.160880,0.168234,-0.228005,-0.022329,-0.041474,0.051025,-0.312522,-0.427488,-0.215221,0.235048,0.062051,0.134062,0.166536,0.193953,0.274553,-0.240456,0.203820,0.215730,0.208659,-0.282242,...,0.436772,-0.282662,-0.320882,0.180086,0.090267,0.113034,-0.337846,0.126446,0.192051,-0.213120,-0.226976,-0.023061,-0.199711,-0.376344,-0.070972,-0.082939,-0.201087,0.086647,-0.162449,-0.067839,0.106283,-0.285938,-0.164674,-0.120791,-0.322989,0.144510,0.127098,-0.252128,-0.058722,-0.217657,0.540956,-0.368099,


<b> Atividade </b>

19) Treine com uma regressão logística

In [ ]:
modelo_embedding = LogisticRegression()
modelo_embedding.fit(X_train_embedding, y_train_embedding)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


<b> Atividade </b>

20) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

#### Calcule as métricas

In [ ]:
y_pred=modelo_embedding.predict(X_test_embedding)

In [ ]:
print(classification_report(y_test_embedding, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.77      0.77      6112
           1       0.78      0.77      0.77      6253

    accuracy                           0.77     12365
   macro avg       0.77      0.77      0.77     12365
weighted avg       0.77      0.77      0.77     12365




<b> Atividade </b>

21) Compare os resultados obtidos com o BagOfWords e com o Embedding. Explique os possíveis motivos desta diferença.

O resultado deu muito diferente se comparar o BagOfWords com o Embedding, porque o Embedding usado durante essa prática foi muito pequeno apenas 100 dimensões, portanto, a quatidade de informação é muito pequena, ou seja, menos valores latentes, impactanto no resultado final. Então, seria necessário aumentar o número de features e as dimensões para melhorar o resultado do Embedding.

# Análise de sentimentos

O modelo que criamos anteriormente é para ilustrar como podemos realizar classificação de documentos.
Quando a tarefa é sobre análise de sentimentos, temos duas opções: treinar nosso próprio modelo, como feito anteriormente ou utilizar uma das inúmeras ferramentas prontas.

Vamos testar as seguintes ferramentas:

- Vader
- Textblob
- Affin

Nesta atividade iremos utilizar as duas variáveis abaixo:

In [ ]:
texto_neg = df.loc[0, "text_en"]
texto_pos = df.loc[49431, "text_en"]

## Vader

<b> Apenas Inglês </b>

O VADER (Valence Aware Dictionary e sEntiment Reasoner) é uma ferramenta de análise de sentimentos baseada em regras e léxico, especificamente identifica os sentimentos expressos nas mídias sociais.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) e (compound score < 0.05)
- negative sentiment: compound score <= -0.05

Mais informações: https://github.com/cjhutto/vaderSentiment


<b> Atividade </b>

22) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto)
```

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
analyzer.polarity_scores(texto_neg)

{'compound': 0.3958, 'neg': 0.126, 'neu': 0.76, 'pos': 0.114}

In [ ]:
analyzer.polarity_scores(texto_pos)

{'compound': 0.9969, 'neg': 0.084, 'neu': 0.737, 'pos': 0.179}

## TextBlob

<b> Apenas inglês </b>

https://www.presentslide.in/2019/08/sentiment-analysis-textblob-library.html

<b> Atividade </b>
 
23) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
sentence=TextBlob(texto)
sentence.sentiment
```

In [ ]:
from textblob import TextBlob

In [ ]:
sentence=TextBlob(texto_neg)

In [ ]:
sentence.sentiment

Sentiment(polarity=0.06385964912280702, subjectivity=0.5629824561403508)

In [ ]:
sentence=TextBlob(texto_pos)

In [ ]:
sentence.sentiment

Sentiment(polarity=0.190819118692253, subjectivity=0.6026226012793177)

## Afinn

- Valor maior que 0 indica sentimento positivo
- Valor menor que 0 indica sentimento negativo

<b> Atividade </b>

24) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
afinn = Afinn()
afinn.score(texto)
```

In [ ]:
from afinn import Afinn

In [ ]:
afinn = Afinn()

In [ ]:
afinn.score(texto_pos)

55.0

In [ ]:
afinn.score(texto_neg)

8.0

<b> Atividade </b>

25) Para você, qual ferramenta teve melhor comportamento?

O primeiro método VADER conseguiu identificar bem p texto positivo, entretanto falhou no texto negativo. O segundo método, TextBlob, varia entre -1 e 1 indo do mais negativo para o mais positivo para a polaridade, apesar de para o texto negativo o valor ser menor que para o texto positivo a diferença entre eles não chega a ser expressiva. O método Afinn acredito que foi o melhor, o valor para o texto positivo ficou alto e apesar do texto negativo não retornar um score negativo a distância entre as duas frases ficou relativamente grande se comparado aos outros métodos.

# Dica:
## Quando for trabalhar com um dataset em inglês, a biblioteca Spacy facilita!

In [ ]:
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import spacy
import pandas as pd

O scpay forne um pacote que já tem série de modelos já treinados em NLP. Inclusive para os embeddings em inglês.

Para mais informações vá em:

https://spacy.io/models/en#en_core_web_md



Com o método abaixo carregamos um dos modelos do spacy:

```python
nlp = spacy.load('en_core_web_md')
```

Para aplicar o modelo, basta passar o texto para o modelo carregado anteriormente:

```python
doc = nlp("This is some text that I am processing with Spacy")
```

Carregue o modelo e imprima doc

In [ ]:
nlp = spacy.load('en_core_web_md')

In [ ]:
doc = nlp("This is some text that I am processing with Spacy")

In [ ]:
doc

This is some text that I am processing with Spacy

In [ ]:
# vetor da primeira palavra
doc[0].vector

array([-8.7595e-02,  3.5502e-01,  6.3868e-02,  2.9292e-01, -2.3635e-01,
       -6.2773e-02, -1.6105e-01, -2.2842e-01,  4.1587e-02,  2.4844e+00,
       -3.8217e-01,  3.2806e-02,  1.2348e-01, -1.8422e-03, -1.3848e-01,
       -1.0005e-03, -4.3081e-02,  1.1659e+00, -4.7327e-02, -5.6004e-02,
        1.5617e-01, -1.3394e-01,  2.3229e-01,  8.7602e-02, -3.2329e-01,
        1.6721e-01, -1.6221e-01, -9.1919e-02, -3.8004e-01,  1.2686e-01,
        6.7819e-02,  3.2509e-01, -5.7245e-02, -3.2630e-01, -1.1903e-01,
       -6.3964e-04, -5.9275e-03, -2.9934e-01, -8.5043e-02, -2.6683e-01,
       -1.5815e-01,  2.5963e-01,  2.2571e-01,  6.2582e-02, -1.9394e-01,
        2.1922e-01, -3.1186e-01,  3.7084e-01, -3.6577e-01, -5.2483e-02,
       -4.3101e-01,  1.2379e-01,  1.5529e-02, -1.2505e-01,  2.2327e-01,
        2.9365e-01, -8.5104e-03, -8.3909e-02,  2.4078e-01, -3.4913e-01,
       -2.8355e-01, -7.6594e-02, -1.7130e-01,  3.2869e-01,  2.9024e-01,
       -6.2741e-02, -5.5278e-02, -2.8706e-01,  7.9608e-02,  1.32

In [ ]:
doc.vector

array([-5.36412969e-02,  2.79353321e-01, -1.05259977e-01, -1.76284965e-02,
        1.34550199e-01,  1.92671806e-01,  5.50469756e-03, -2.39132687e-01,
       -4.06342074e-02,  1.78010297e+00, -1.80772960e-01,  1.02661893e-01,
        6.84069991e-02, -5.09319194e-02, -7.65837058e-02, -3.77540514e-02,
        8.24129581e-03,  1.37752008e+00, -1.78934380e-01, -5.76109104e-02,
        1.66338980e-02, -3.62196006e-02, -7.48579949e-02,  4.40651290e-02,
       -2.65241470e-02,  2.41529979e-02,  9.79370065e-03, -1.13990309e-03,
        1.59522101e-01, -1.56648397e-01, -9.12139937e-02,  9.11872908e-02,
        1.07169405e-01, -1.08843103e-01, -7.94988051e-02, -4.74919155e-02,
       -1.60613850e-01, -2.82304995e-02, -1.03425637e-01, -1.14933215e-01,
        1.62531182e-01, -1.01342008e-01,  2.17013666e-03,  3.47881988e-02,
       -6.34927005e-02,  2.44374484e-01, -3.01910043e-02, -1.46046979e-02,
       -1.06488302e-01,  6.26319647e-03, -1.30655810e-01,  7.04905912e-02,
       -4.86716032e-02,  

Ao aplicar o modelo carregado a variável <b> doc </b> já possui os embeddings de cada uma das palavras e o embedding da frase, que é a média de todos vetores de todas palavras.

```python
#vetor da primeira palavra
doc[0].vector
#vetor agregado pela média - embedding do documento
doc.vector
```
O código abaixo mostra que a média de uma posição em específico dos embeddings de todas as palavras e a posição do embedding do documento possuem o mesmo valor.

In [ ]:
def calcula_media_posicao(x):
    soma = 0
    vector = []
    for i in range(0,len(doc)):
        vector.append(doc[i].vector)    
    
    for v in vector:
        soma += v[x]
    return soma/len(doc)

In [ ]:
round(calcula_media_posicao(10),6)

-0.180773

In [ ]:
round(doc.vector[10], 6)

-0.180773